In [1]:
#!/usr/bin/env python3
import os
import re
import sys
import collections
import argparse
import tables
import glob
import itertools
import math

import numpy as np
import pandas as pd
import scipy.stats as stats
import scipy.sparse as sp_sparse

from multiprocessing import Pool
from collections import defaultdict
from scipy import sparse, io
from scipy.sparse import csr_matrix
from multiprocessing import Pool

In [2]:
def load_data(data_dir, region):
    region = region.replace(':', '-')
    up_pval_file   = data_dir + region + '-up_log-pval.mat'
    down_pval_file = data_dir + region + '-down_log-pval.mat'

    fc_files = glob.glob(data_dir + region + '*-foldchange.mat')
    if len(fc_files) == 1:
        fc_file = fc_files[0]
    else:
        print("In the following region:" + region)
        print('FC file is not unique.')
        sys.exit()
    
    pval_list_up = io.loadmat(up_pval_file)['matrix'][0]
    pval_list_down = io.loadmat(down_pval_file)['matrix'][0]
    fc = io.loadmat(fc_file)['matrix'][0]
    
    num_sgrna_cell = int(fc_file.split('/')[-1].split('-')[2])
    return pval_list_up, pval_list_down, fc, num_sgrna_cell

In [3]:
#Manhatthan Plot for up- and down-regulated genes separately
#get the FDR by using Benjamin-Hochberg methods
def pval_calculator(fc, annot_df, pval_list_up, pval_list_down):
    up_idx        = np.where(np.array(fc) > 1)[0]
    down_idx      = np.where(np.array(fc) < 1)[0]
    plot_down_idx = np.intersect1d(down_idx, annot_df.idx)
    plot_up_idx   = np.intersect1d(up_idx, annot_df.idx)

    #preprocess the pvals for plotting
    pval_list_up[np.argwhere(pval_list_up == 0)] = 1
    pval_list_up[np.isinf(pval_list_up)] = 0

    pval_list_down[np.argwhere(pval_list_down == 0)] = 1
    pval_list_down[np.isinf(pval_list_down)] = 0

    #calculate the p_value for plotting
    combined_p_val = [0] * (len(fc))

    for i in up_idx:
        if (-1*pval_list_up[i] - unordered_up_percent_list[i] > 0):
            combined_p_val[i] = (-1*pval_list_up[i] - unordered_up_percent_list[i]) / np.log(10)
        else:
            combined_p_val[i] = 0

    for i in down_idx:

        if (pval_list_down[i] + unordered_down_percent_list[i] < 0):
            combined_p_val[i] = (pval_list_down[i] + unordered_down_percent_list[i]) / np.log(10)
        else:
            combined_p_val[i] = 0
        
    combined_p_val=np.array(combined_p_val)
    return combined_p_val

In [4]:
def get_distance(region, gene_pos, annot_df):
    length_list = [
        0, 248956422,491149951,689445510,879660065,1061198324,               
        1232004303,1391350276,1536488912,1674883629,1808681051,
        1943767673,2077042982,2191407310,2298451028,2400442217,
        2490780562,2574038003,2654411288,2713028904,2777473071,
        2824183054,2875001522,3031029399
    ]
    
    chr_order = [
        'chr1', 'chr2', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chr8', 
        'chr9', 'chr10', 'chr11', 'chr12', 'chr13', 'chr14', 'chr15',
        'chr16', 'chr17', 'chr18', 'chr19', 'chr20', 'chr21', 'chr22', 'chrX'
    ]
    
    enh_chrom, left, right = re.split('[:|-]', region)
    dist =  int(gene_pos) - int(left) - length_list[chr_order.index(enh_chrom)]
    
    return np.absolute(dist)

In [5]:
#load the annotation information
working_dir = '../Data/'

length_list = [0, 248956422,491149951,689445510,879660065,1061198324,
               1232004303,1391350276,1536488912,1674883629,1808681051,
               1943767673,2077042982,2191407310,2298451028,2400442217,
               2490780562,2574038003,2654411288,2713028904,2777473071,
               2824183054,2875001522,3031029399]
    
chr_order = ['chr1', 'chr2', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chr8', 
             'chr9', 'chr10', 'chr11', 'chr12', 'chr13', 'chr14', 'chr15',
             'chr16', 'chr17', 'chr18', 'chr19', 'chr20', 'chr21', 'chr22', 'chrX']

annot_df = pd.read_csv(working_dir + 'Annotations/plot_annotation.txt',
                       header = None, 
                       sep='\t',
                       names = ["idx", "gene_names", "chromosome", "pos", "strand", "color_idx", "chr_idx"])

In [6]:
#load the background p-val for multiple hypothesis test
#percent_dir = '/project/GCRB/Hon_lab/s160875/03.analysis/Mosaic-seq/CROP-DE-analysis_10X-66K_no_downsampling-CPM.hg38/\
#rand_sgrna/hypergeo_test_metacell_rand-sgrna/'

percent_dir = working_dir + 'Gene_cutoff/'

up_percent_file = percent_dir + 'Perct_99.9_combined_cutoff.up_genes.mat'
unordered_up_percent_list = np.asarray(io.loadmat(up_percent_file)['matrix']).squeeze()
down_percent_file = percent_dir + 'Perct_99.9_combined_cutoff.down_genes.mat'
unordered_down_percent_list = np.asarray(io.loadmat(down_percent_file)['matrix']).squeeze()

In [7]:
#load local hits df
local_gene_df = pd.read_pickle(working_dir + '/Hits_calling/local_hits_99.9.pkl')

In [8]:
#load data
data_dir = working_dir + 'Hypergeometric_pvals/'

global_hits_df = pd.DataFrame(columns=[
    'idx', 'gene_names', 'chromosome', 
    'pos', 'strand', 'chr_idx', 'region', 'log2fc',
    '-log10padj',
    ])
    
for region in np.unique(local_gene_df.region.values):
    pval_list_up, pval_list_down, fc, num_sgrna_cell = load_data(data_dir, region)
    enh_chrom, left, right = re.split(r'[:-]+', region)
    combined_p_val = pval_calculator(fc, annot_df, pval_list_up, pval_list_down)
    
    #get the outlier
    raw_outliers = []
    counter = 0
    for i,e in enumerate(combined_p_val):
        if (e < -1.9 or e > 2.8):
            raw_outliers.append(counter)
        counter += 1
    outliers = np.intersect1d(raw_outliers, annot_df.idx)

    #if there is any DE gene, output the csv file containing all the significant genes
    if any(outliers):
        for i in outliers:
            chrom, left, right = re.split('[:|-]', region)
            
            hits_gene_series = annot_df.loc[annot_df.idx == i].iloc[0,:]
            dist = get_distance(region, hits_gene_series['pos'], annot_df)  
            
            if (chrom == hits_gene_series.chromosome) & (dist < 2e6):
                continue

            hits_gene_series['distance'] = dist
            hits_gene_series['log2fc'] = np.log2(fc[i])
            if fc[i] > 1:
                hits_gene_series['-log10padj'] = combined_p_val[i]
            elif fc[i] < 1:
                hits_gene_series['-log10padj'] = -1*combined_p_val[i]
                
            hits_gene_series['region'] = region
            global_hits_df = global_hits_df.append(hits_gene_series)

In [9]:
global_hits_df.head()

,idx,gene_names,chromosome,pos,strand,chr_idx,region,log2fc,-log10padj,color_idx,distance
52823,52823,ELMO2,chr20,2759461889,-,19,chr10:61765677-61766077,-1.469902,2.146788,1.0,1.022813e+09
27865,27865,RPL7A,chr9,1669837126,+,8,chr10:72320697-72321097,-0.076568,2.925367,0.0,7.736720e+07
47558,47558,CTD-2561B21.5,chr17,2571751775,-,16,chr10:72320697-72321097,1.968059,2.818604,0.0,8.245474e+08
835,835,SH3BGRL3,chr1,26279176,+,0,chr11:33881869-33882269,0.400005,3.385772,0.0,1.816284e+09
55000,55000,LGALS1,chr22,2861858662,+,21,chr11:33881869-33882269,0.398439,4.143693,1.0,1.019296e+09


In [10]:
#global_hits_df.to_csv('./global_hits.csv')
#global_hits_df.to_pickle('./global_hits.pkl')

## Construct Network

In [11]:
import networkx as nx

In [12]:
G = nx.DiGraph()
for r in np.unique(local_gene_df.region.values):
    
    primary_genes = local_gene_df.loc[local_gene_df.region == r].gene_names.values
    secondary_genes = global_hits_df.loc[global_hits_df.region == r].gene_names.values
    G.add_node(r, type='enhancer', color=0)
    G.add_nodes_from(primary_genes, type='gene', color=1)
    G.add_nodes_from(secondary_genes, type='gene', color=1)
    
    #add edges
    for pg in primary_genes:
        G.add_edge(r, pg, regulation='blue')

    log2fc_list = global_hits_df.loc[global_hits_df.region == r].log2fc.values
    for (pg, sg) in itertools.product(primary_genes, secondary_genes):
        sg_fc = global_hits_df.loc[(global_hits_df.region == r) & (global_hits_df.gene_names == sg)].log2fc.values
        if sg_fc > 0:
            G.add_edge(pg, sg, regulation='red')
        else:
            G.add_edge(pg, sg, regulation='blue')

In [19]:
nx.nx.write_gml(G, './network_K562.gml')

## Visualize Network with Force Atlas 2 Layout

In [13]:
#To get the largest network, we first convert the graph to undrected
undi_G = G.to_undirected()

#Now we keep the largest sub-network
largest_component = max(nx.connected_components(undi_G), key=len)
sub_G = undi_G.subgraph(largest_component)
sub_G = nx.Graph(sub_G)

#for visualization purposes, we remove several nodes
nodes_removal = ['AHI1', 'KRT8', 'TRIM44', 'NAT10']
for n in nodes_removal:
    sub_G.remove_node(n)

In [14]:
from fa2 import ForceAtlas2
import matplotlib.pyplot as plt

forceatlas2 = ForceAtlas2(
                        # Behavior alternatives
                        outboundAttractionDistribution=False,  # Dissuade hubs
                        linLogMode=False,  # NOT IMPLEMENTED
                        adjustSizes=False,  # Prevent overlap (NOT IMPLEMENTED)
                        edgeWeightInfluence=1,
                        # Performance
                        jitterTolerance=1.0,  # Tolerance
                        barnesHutOptimize=True,
                        barnesHutTheta=1.2,
                        multiThreaded=False,  # NOT IMPLEMENTED
                        # Tuning
                        scalingRatio=1,
                        strongGravityMode=False,
                        gravity=5,
                        # Log
                        verbose=False
)

positions = forceatlas2.forceatlas2_networkx_layout(sub_G, pos=None, iterations=2000)

In [15]:
enh_node_list = [i for i in sub_G.node() if i.startswith('chr')]
gene_node_list = [i for i in sub_G.node() if not i.startswith('chr')]
large_node_list = [n for n, d in sub_G.degree() if d > 20] #get nodes for key TFs

In [16]:
node_degree = np.array([float(sub_G.degree(v)) for v in sub_G if not v.startswith('chr')])
large_node_size = np.array([d for n, d in sub_G.degree() if d > 20]) 
node_degree = np.clip(node_degree, 2, 10) #set the limit for plotting

In [20]:
plt.figure(figsize=(20,20))

#produce the curved edges
from matplotlib.collections import LineCollection
from curved_edges import curved_edges
curves = curved_edges(sub_G, positions)
lc = LineCollection(curves, color='#C7C7C7')

plt.gca().add_collection(lc)

#plot all nodes
all_nodes = nx.draw_networkx_nodes(sub_G, positions, 
                                   nodelist = gene_node_list, 
                                   node_shape = 'o',
                                   with_label=True,
                                   linewidth=2,
                                   node_size=node_degree*60, with_labels=False,
                                   node_color=node_degree,
                                   cmap='Pastel1',
                                   edge_color='k',
                                   #                       node_color='#2278B5',
                                   #                       alpha=0.5
                                  )
all_nodes.set_edgecolor('k')

large_nodes = nx.draw_networkx_nodes(sub_G, positions,                       
                                     nodelist = large_node_list, 
                                     node_shape = 'o',
                                     labels=large_node_list,
                                     linewidth=2,
                                     node_size=2000, 
                                     node_color='#D62A28',
                                     edge_color='k'
                                    )
large_nodes.set_edgecolor('k')

nx.draw_networkx_nodes(sub_G, positions, 
                       nodelist = enh_node_list,
                       node_shape = 'D',
                       node_size=80, with_labels=True, node_color='k')

#plot labels for key transctiptional factors (large nodes)
labels = {}
for i in large_node_list:
    labels[i] = i
nx.draw_networkx_labels(sub_G, positions, labels, font_size=16)

plt.axis('off')
plt.gca().set_facecolor('k')
plt.savefig('./K562_network.png', dpi=300)
plt.show()